### <center>Class 10: Multiple Linear Regression</center>

In [ ]:
import os
import sys
import warnings
from typing import List
import copy

import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import math
# from patsy import dmatrices
from stargazer.stargazer import Stargazer
from utils import lspline

import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline

## Data - Hotels

In [ ]:
path = os.path.join(os.pardir, 'data', 'hotels-vienna.csv') # this will produce a path with the right syntax for your operating system
path

In [ ]:
df_hotels = pd.read_csv(path)

In [ ]:
df_hotels

In [ ]:
df_hotels = df_hotels[
    (df_hotels.accommodation_type == 'Hotel')
    & (df_hotels.city_actual == 'Vienna')
    & (df_hotels.stars >= 3)
    & (df_hotels.stars <= 4)
    & (df_hotels.price <= 600)]


In [ ]:
df_hotels.shape

In [ ]:
df_hotels["lnprice"] = np.log(df_hotels["price"])
df_hotels["distance2"] = df_hotels["distance"]
df_hotels.loc[df_hotels["distance2"] < 0.05, "distance2"] = 0.05 # making sure that the log transformation is feasible
df_hotels["lndistance"] = np.log(df_hotels["distance2"])
df_hotels["star35"] = df_hotels["stars"] == 3.5
df_hotels["star4"] = df_hotels["stars"] == 4

In [ ]:
df_hotels[["distance", "price", "lnprice"]].describe().T.round(2)

## Multiple Linear Regressions

**rating, distance, and both**

In [ ]:
reg0 = smf.ols("lnprice ~ rating", data = df_hotels).fit(cov_type = 'HC0')
reg1 = smf.ols("lnprice ~ distance", data = df_hotels).fit(cov_type = 'HC0')
reg2 = smf.ols("lnprice ~ distance + rating", data = df_hotels).fit(cov_type = 'HC0')

In [ ]:
Stargazer([reg0, reg1, reg2])

The `dmatrices` method in the `Patsy` package is used to create design matrices for statistical models. It takes a formula string and a data frame as input and returns two matrices: one for the dependent variable(s) and one for the independent variable(s). This is particularly useful for preparing data for regression analysis or other statistical modeling.

**distance and rating with splines + stars**

In [ ]:
reg3 = smf.ols(
    "lnprice ~ lspline(distance,[1,4]) + lspline(rating, 3.5) + star35 + star4",
    data = df_hotels,
).fit(cov_type = 'HC0')

In [ ]:
print(reg3.summary())

**distance with splines only**

In [ ]:
reg4 = smf.ols("lnprice ~ lspline(distance,[1,4])", data = df_hotels).fit(cov_type = 'HC0')

In [ ]:
Stargazer([reg0, reg1, reg2, reg3, reg4])

**Question**: How can we interpret the rating splines and the star dummies? How are they related?

#### Finding best deals

In [ ]:
df_hotels['lnprice_hat'] = reg3.predict(df_hotels)

In [ ]:
pd.DataFrame({'a': df_hotels['lnprice_hat'], 'b':reg3.fittedvalues})

In [ ]:
plt.figure(figsize = (6,6))
sns.scatterplot(data = df_hotels, x = 'lnprice_hat', y = 'lnprice')
plt.plot(df_hotels.lnprice_hat, df_hotels.lnprice_hat, color = 'k')
plt.xlabel('predicted price')
plt.ylabel('actual price')
plt.title('Predicted vs actual prices in Vienna');

The best deal is where the model overestimates the most, aka where the `residuals` are the largest in abosulte value. But how are the residuals calculated?

In [ ]:
df_hotels['residuals'] = reg3.resid

In [ ]:
df_hotels[['lnprice', 'lnprice_hat', 'residuals']].iloc[0:10]

As we can see 
\begin{equation}
\text{residuals = actual - predicted}
\end{equation}

So the best deal is...

In [ ]:
df_best_deal = df_hotels[df_hotels.residuals == df_hotels.residuals.min()]
df_best_deal.T

In [ ]:
plt.figure(figsize = (6,6))
# predicted vs actual prices
sns.scatterplot(data = df_hotels, x = 'lnprice_hat', y = 'lnprice')
# 45-degree line to separate over- and underpriced datapoints
plt.plot(df_hotels.lnprice_hat, df_hotels.lnprice_hat, color = 'k')
plt.xlabel('predicted price')
plt.ylabel('actual price')
# identifying the best deal
plt.plot(df_best_deal.lnprice_hat, df_best_deal.lnprice, marker = 'o', color = 'indianred') # add an extra point to cover the best deal
# aesthetics
# plt.xlim([3.75, 6])
# plt.ylim([3.75, 6])
plt.text(x= df_best_deal.lnprice_hat + 0.05, y = df_best_deal.lnprice, s = 'best deal', color = 'indianred')
plt.title('Predicted vs actual prices in Vienna \nand the best deal');

In [ ]:
df_hotels.sort_values(by = 'distance', inplace = True, ascending= True)

Once we have multiple independent variables we rarely plot actual and predicted against one dependent variable in a scatterplot. For demonstration pruposes, however, it makes sense to go back and plot our results in the $lnprice$ vs $distance$ space once more, since distance is probably an important variable for the user. 

In [ ]:
fig, ax = plt.subplots(figsize = (7,5))
ax.scatter(df_hotels.distance, df_hotels.lnprice, color = 'royalblue', s = 10, label = 'actual')
ax.scatter(df_hotels.distance, df_hotels.lnprice_hat, color = 'k', s = 20, label = 'predicted')
plt.legend(labelcolor = ['royalblue', 'k'])
plt.xlabel('distance in miles')
plt.ylabel('log price')
plt.title('Actual vs predicted prices in Vienna');

## Data - Earnings

In [ ]:
path = os.path.join(os.pardir, 'data', 'morg-2014-emp.csv') # this will produce a path with the right syntax for your operating system
path

In [ ]:
df_earnings = pd.read_csv(path, index_col = 0)

In [ ]:
df_earnings.head()

In [ ]:
df_earnings.shape

In [ ]:
df_earnings.info()

In [ ]:
df_earnings = df_earnings.query("uhours >= 20 & earnwke > 0 & age >= 24 & age <= 64 & grade92 >= 44")

In [ ]:
df_earnings.shape

In [ ]:
df_earnings.sort_values(by = 'age', inplace = True) # for plotting

#### Feature engineering + EDA

In [ ]:
df_earnings.sex.value_counts()

In [ ]:
df_earnings["female"] = (df_earnings.sex == 2).astype(int)
df_earnings["w"] = df_earnings["earnwke"] / df_earnings["uhours"]
df_earnings["lnw"] = np.log(df_earnings["w"])
df_earnings['gender'] = df_earnings.sex.map(lambda x: 'female' if x == 2 else 'male') # for plotting

In [ ]:
df_earnings.gender.value_counts()

In [ ]:
df_earnings.loc[:, ["earnwke", "uhours", "w"]].describe().T

Visualization of gender distribution.

`seaborn`

In [ ]:
g = sns.FacetGrid(data = df_earnings, col= "gender", height= 5, aspect = 0.9)
g.map(sns.histplot, "age", bins = range(25, 70, 5), shrink = 0.9)
plt.suptitle('Age distribution by sex');

`matplotlib`

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols = 2, figsize = (10,5), sharey= True)
ax1.hist(df_earnings[df_earnings.gender == 'male'].age, bins = range(25, 70, 5), rwidth = 0.9, edgecolor = 'k')
ax1.set_title('males')
ax1.set_xticks(range(25, 70, 5))
ax1.set_ylabel('Count')
ax1.set_xlabel('age')
ax2.hist(df_earnings[df_earnings.gender == 'female'].age, bins = range(25, 70, 5), rwidth = 0.9, edgecolor = 'k')
ax2.set_title('females')
ax2.set_xticks(range(25, 70, 5))
ax2.set_xlabel('age')
fig.suptitle('Age distribution by sex');

In [ ]:
sns.kdeplot(data = df_earnings, x = 'age', hue = 'gender', palette = ['k', 'royalblue'], )
plt.xticks(range(20, 75, 5))
plt.title('Age distribution: males vs females');

## Regressions

**log earnings, gender, age**

In [ ]:
reg0 = smf.ols(formula = "lnw ~ female", data = df_earnings).fit(cov_type="HC1")
reg1 = smf.ols(formula="lnw ~ female + age", data = df_earnings).fit(cov_type="HC1")
reg2 = smf.ols(formula = "age  ~female", data = df_earnings).fit(cov_type="HC1")

**Note**: on heteroscedasticity-consistent standard errorrs see this: https://en.wikipedia.org/wiki/Heteroskedasticity-consistent_standard_errors or this: https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html

In [ ]:
stargazer = Stargazer([reg0, reg1, reg2])
stargazer.custom_columns(["ln wage", "ln wage", "age"], [1, 1, 1])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer

**Question**: How would you interpret the constant?

**log wage vs age in various function forms**

In [ ]:
df_earnings["agesq"] = np.power(df_earnings["age"], 2)
df_earnings["agecu"] = np.power(df_earnings["age"], 3)
df_earnings["agequ"] = np.power(df_earnings["age"], 4)

In [ ]:
reg3 = smf.ols(formula="lnw ~ female", data=df_earnings).fit(cov_type="HC1")
reg4 = smf.ols(formula="lnw ~ female + age", data=df_earnings).fit(cov_type="HC1")
reg5 = smf.ols(formula="lnw ~ female + age + agesq", data=df_earnings).fit(cov_type="HC1")
reg6 = smf.ols(formula="lnw ~ female + age + agesq + agecu + agequ", data=df_earnings).fit(cov_type="HC1")

In [ ]:
stargazer = Stargazer([reg3, reg4, reg5, reg6])
stargazer.covariate_order(["Intercept", "female", "age", "agesq", "agecu", "agequ"])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer

**log earnings, gender and education**

In [ ]:
df_earnings["ed_MA"] = (df_earnings["grade92"] == 44).astype(int)
df_earnings["ed_Profess"] = (df_earnings["grade92"] == 45).astype(int)
df_earnings["ed_Phd"] = (df_earnings["grade92"] == 46).astype(int)

In [ ]:
reg7 = smf.ols(formula="lnw ~ female", data=df_earnings).fit(cov_type="HC1")
reg8 = smf.ols(formula="lnw ~ female + ed_Profess + ed_Phd", data=df_earnings).fit(cov_type="HC1")
reg9 = smf.ols(formula="lnw ~ female + ed_Profess + ed_MA", data=df_earnings).fit(cov_type="HC1")

In [ ]:
stargazer = Stargazer([reg7, reg8, reg9])
stargazer.covariate_order(["Intercept", "female", "ed_Profess", "ed_Phd", "ed_MA"])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer

**log earnings, gender, age, and their interaction**

In [ ]:
reg10 = smf.ols(formula="lnw ~ age", data=df_earnings.query("female==1")).fit(cov_type="HC1")
reg11 = smf.ols(formula="lnw ~ age", data=df_earnings.query("female==0")).fit(cov_type="HC1")
reg12 = smf.ols(formula="lnw ~ female + age + age  *female", data=df_earnings).fit(cov_type="HC1")

In [ ]:
stargazer = Stargazer([reg10, reg11, reg12])
stargazer.covariate_order(["Intercept", "female", "age", "age:female"])
stargazer.rename_covariates({"Intercept": "Constant", "age:female": "female x age"})
stargazer.custom_columns(["Women", "Men", "All"], [1, 1, 1])
stargazer

**gender-age interactions on steroids**

In [ ]:
reg13 = smf.ols(formula="lnw ~ age + agesq + agecu + agequ", data=df_earnings.query("female==1")).fit(
    cov_type="HC1"
)
reg14 = smf.ols(formula="lnw ~ age + agesq + agecu + agequ", data=df_earnings.query("female==0")).fit(
    cov_type="HC1"
)
reg15 = smf.ols(
    formula="lnw ~ age + agesq + agecu + agequ + female + female*age + female*agesq + female*agecu + female*agequ",
    data=df_earnings,
).fit(cov_type="HC1")

In [ ]:
stargazer = Stargazer([reg13, reg14, reg15])
stargazer.custom_columns(["Women", "Men", "All"], [1, 1, 1])
stargazer

#### Prediction of the general pattern between males and females

We are using `reg12`, where $age$ is interacted with $sex$. Why are we using reg12 and not reg15?

**males**

In [ ]:
df_earnings_m = df_earnings.query("female==0")
df_earnings_m

In [ ]:
pred = reg12.get_prediction(df_earnings_m).summary_frame()[['mean', 'mean_ci_lower', 'mean_ci_upper']]

In [ ]:
pred.rename(columns={'mean':'fit'}, inplace=True)

In [ ]:
pred

In [ ]:
df_earnings_m = df_earnings_m.reset_index(drop = True).join(pred)

In [ ]:
df_earnings_m.tail()

In [ ]:
df_earnings_f = df_earnings.query("female==1")
pred = reg12.get_prediction(df_earnings_f).summary_frame()[['mean', 'mean_ci_lower', 'mean_ci_upper']]
pred.rename(columns={'mean':'fit'}, inplace=True)
df_earnings_f = df_earnings_f.reset_index(drop = True).join(pred)

In [ ]:
fig, ax = plt.subplots(figsize = (7,5))
# males
ax.plot(df_earnings_m.age, df_earnings_m.fit, color = 'k', label = 'males')
ax.fill_between(df_earnings_m.age, df_earnings_m.mean_ci_lower, df_earnings_m.mean_ci_upper, color = 'k', alpha = 0.4)
# females
ax.plot(df_earnings_f.age, df_earnings_f.fit, color = 'indianred', label = 'females')
ax.fill_between(df_earnings_f.age, df_earnings_f.mean_ci_lower, df_earnings_f.mean_ci_upper, color = 'indianred', alpha = 0.4)
# aesthetics
ax.legend(labelcolor = ['k', 'indianred'])
ax.grid(linestyle='dotted')
ax.set_ylim([2.8, 3.8])
ax.set_xlabel('years')
ax.set_ylabel('log(hourly earnings, USD)')
ax.set_title('Expected log wage as a linear function of age - males and females');

Same exercise, using `reg15`, the 'best fit' model.

In [ ]:
df_earnings_m = df_earnings.query("female==0")
pred = reg15.get_prediction(df_earnings_m).summary_frame()[['mean', 'mean_ci_lower', 'mean_ci_upper']]
pred.rename(columns={'mean':'fit'}, inplace=True)
df_earnings_m = df_earnings_m.reset_index(drop = True).join(pred)

In [ ]:
df_earnings_f = df_earnings.query("female==1")
pred = reg15.get_prediction(df_earnings_f).summary_frame()[['mean', 'mean_ci_lower', 'mean_ci_upper']]
pred.rename(columns={'mean':'fit'}, inplace=True)
df_earnings_f = df_earnings_f.reset_index(drop = True).join(pred)

In [ ]:
fig, ax = plt.subplots(figsize = (7,5))
# males
ax.plot(df_earnings_m.age, df_earnings_m.fit, color = 'k', label = 'males')
ax.fill_between(df_earnings_m.age, df_earnings_m.mean_ci_lower, df_earnings_m.mean_ci_upper, color = 'k', alpha = 0.4)
# females
ax.plot(df_earnings_f.age, df_earnings_f.fit, color = 'indianred', label = 'females')
ax.fill_between(df_earnings_f.age, df_earnings_f.mean_ci_lower, df_earnings_f.mean_ci_upper, color = 'indianred', alpha = 0.4)
# aesthetics
ax.legend(labelcolor = ['k', 'indianred'])
ax.grid(linestyle='dotted')
ax.set_ylim([2.8, 3.8])
ax.set_xlabel('years')
ax.set_ylabel('log(hourly earnings, USD)')
ax.set_title('Expected log wage as a non-linear function of age - males and females');

#### Extended regression: all variables to find the marginal impact of gender

In [ ]:
df_earnings = df_earnings.query("age >= 40 & age <= 60")

In [ ]:
df_earnings["white"] = (df_earnings["race"] == 1).astype(int)
df_earnings["afram"] = (df_earnings["race"] == 2).astype(int)
df_earnings["asian"] = (df_earnings["race"] == 4).astype(int)
df_earnings["hisp"] = (df_earnings["ethnic"].notna()).astype(int)
df_earnings["othernonw"] = (
    (df_earnings["white"] == 0) & (df_earnings["afram"] == 0) & (df_earnings["asian"] == 0) & (df_earnings["hisp"] == 0)
).astype(int)
df_earnings["nonUSborn"] = (
    (df_earnings["prcitshp"] == "Foreign Born, US Cit By Naturalization")
    | (df_earnings["prcitshp"] == "Foreign Born, Not a US Citizen")
).astype(int)

In [ ]:
# Potentially endogeneous demographics
df_earnings["married"] = ((df_earnings["marital"] == 1) | (df_earnings["marital"] == 2)).astype(int)
df_earnings["divorced"] = ((df_earnings["marital"] == 3) & (df_earnings["marital"] == 5)).astype(int)
df_earnings["wirowed"] = (df_earnings["marital"] == 4).astype(int)
df_earnings["nevermar"] = (df_earnings["marital"] == 7).astype(int)

df_earnings["child0"] = (df_earnings["chldpres"] == 0).astype(int)
df_earnings["child1"] = (df_earnings["chldpres"] == 1).astype(int)
df_earnings["child2"] = (df_earnings["chldpres"] == 2).astype(int)
df_earnings["child3"] = (df_earnings["chldpres"] == 3).astype(int)
df_earnings["child4pl"] = (df_earnings["chldpres"] >= 4).astype(int)

# Work-related variables
df_earnings["fedgov"] = (df_earnings["class"] == "Government - Federal").astype(int)
df_earnings["stagov"] = (df_earnings["class"] == "Government - State").astype(int)
df_earnings["locgov"] = (df_earnings["class"] == "Government - Local").astype(int)
df_earnings["nonprof"] = (df_earnings["class"] == "Private, Nonprofit").astype(int)
df_earnings["ind2dig"] = ((pd.Categorical(df_earnings["ind02"]).codes + 1) / 100).astype(int)
df_earnings["occ2dig"] = (df_earnings["occ2012"] / 100).astype(int)
df_earnings["union"] = ((df_earnings["unionmme"] == "Yes") | (df_earnings["unioncov"] == "Yes")).astype(int)

In [ ]:
df_earnings["uhourssq"] = np.power(df_earnings["uhours"], 2)
df_earnings["uhourscu"] = np.power(df_earnings["uhours"], 3)
df_earnings["uhoursqu"] = np.power(df_earnings["uhours"], 4)

In [ ]:
df_earnings.stfips

In [ ]:
reg1 = smf.ols(formula="lnw ~ female", data=df_earnings).fit(cov_type="HC1")

reg2 = smf.ols(
    formula="lnw ~ female + age + ed_Profess + ed_Phd", data=df_earnings
).fit(cov_type="HC1")

reg3 = smf.ols(
    formula="lnw ~ female + age + afram + hisp + asian + othernonw + nonUSborn + ed_Profess + ed_Phd + married + divorced+ wirowed + child1 + child2 + child3 +child4pl + C(stfips) + uhours + fedgov + stagov + locgov + nonprof + union + C(ind2dig) + C(occ2dig)",
    data=df_earnings,
).fit(cov_type="HC1")

reg4 = smf.ols(
    formula="lnw ~ female + age + afram + hisp + asian + othernonw + nonUSborn + ed_Profess + ed_Phd + married + divorced+ wirowed + child1 + child2 + child3 +child4pl + C(stfips) + uhours + fedgov + stagov + locgov + nonprof + union + C(ind2dig) + C(occ2dig) + agesq + agecu + agequ + uhoursqu + uhourscu + uhourssq",
    data=df_earnings,
).fit(cov_type="HC1")

In [ ]:
print(reg3.summary())

In [ ]:
print(reg4.summary())

In [ ]:
reg4.pvalues.where(lambda x: x < 0.05).dropna()

## Summar findings

The following simplified Stargazer output summarizes the models built. Only the `female` variable is shown with parameter values. Other variables are grouped into logical categories and indicated whether they are included in the appropriate model. 

In [ ]:
stargazer = Stargazer([reg1, reg2, reg3, reg4])
stargazer.covariate_order(["female"])
stargazer.add_line("Age and education", ["", "Yes", "Yes", "Yes"])
stargazer.add_line("Family circumstances", ["", "", "Yes", "Yes"])
stargazer.add_line("Demographic background", ["", "", "Yes", "Yes"])
stargazer.add_line("Job characteristics", ["", "", "Yes", "Yes"])
stargazer.add_line("Age in polynomial", ["", "", "", "Yes"])
stargazer.add_line("Hours in polynomial", ["", "", "", "Yes"])
stargazer